# Installation & Setup

In [83]:
%%capture
!pip install nltk
!pip install transformers
!pip install datasets
!pip install srt
!pip install gdown
!apt install ffmpeg
!pip install deepmultilingualpunctuation
!pip install silero-vad
!pip install spacy
!pip install pytextrank
!pip install pydub

!pip install git+https://github.com/m-bain/whisperX.git

In [95]:
import os
import numpy as np
import pandas as pd
import tarfile
import gdown
import nltk
from nltk.tokenize import sent_tokenize
import spacy
import srt
from functools import reduce
from pydub import AudioSegment

from datasets import load_dataset
import torch
import torchaudio
import torch.nn.functional as F
from transformers import \
LongformerTokenizer, LongformerModel, LongformerForSequenceClassification, \
RobertaTokenizer, RobertaForTokenClassification, Trainer, TrainingArguments, \
LEDTokenizer, LEDForConditionalGeneration

# Text
import pytextrank

# Audio
import whisperx
import silero_vad
from silero_vad import load_silero_vad, read_audio, get_speech_timestamps

# Video

In [3]:
%%capture
# NLTK
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [4]:
# Notebook config
os.environ["WANDB_DISABLED"] = "true"

# Variables

In [5]:
video_input = "dataset/teamwork in the classroom.mov"

full_base = os.path.dirname(video_input)
path_dataset = full_base
filename = os.path.basename(video_input)
filename_without_extension = os.path.splitext(filename)[0]

filename_video_input = filename
filename_subtitles_output = filename_without_extension + ".srt"
filename_audio_output = filename_without_extension + ".wav"

subtitles_output = os.path.join(full_base, filename_subtitles_output)
audio_output = os.path.join(full_base, filename_audio_output)

video = ''
audio = ''
subtitles = ''
sentences = ''

In [6]:
print(audio_output)

dataset/teamwork in the classroom.wav


# Datasets

In [7]:
# Google Drive Dataset Location
folder_id = '1k7DLJPl1xz9lpU4l3dZYtPe1XawhrXeC' # taken from drive.google.com/drive/u/1/folders/1k7D...(this part)
gdown.download_folder(id=folder_id, quiet=False, use_cookies=False)

Retrieving folder contents


Processing file 1vuIW3CVm2p_Ig-_srJ5sIwUxzTGqdGHp assessing students without exams.mov
Processing file 1OP3zzSmpKJ0RDPasl9AGQD2yNeCEXtoR flipped learning basics.mov
Processing file 1wslcvTNd88FQMXJgvGbKR3sjwXORR6xt teamwork in the classroom.mov


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1vuIW3CVm2p_Ig-_srJ5sIwUxzTGqdGHp
From (redirected): https://drive.google.com/uc?id=1vuIW3CVm2p_Ig-_srJ5sIwUxzTGqdGHp&confirm=t&uuid=19df07d6-2d86-4e16-a7fe-66bf3650c43d
To: /content/dataset/assessing students without exams.mov
100%|██████████| 875M/875M [00:11<00:00, 78.2MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1OP3zzSmpKJ0RDPasl9AGQD2yNeCEXtoR
From (redirected): https://drive.google.com/uc?id=1OP3zzSmpKJ0RDPasl9AGQD2yNeCEXtoR&confirm=t&uuid=956ab9f8-6d62-4814-b5f8-fd72055b1f65
To: /content/dataset/flipped learning basics.mov
100%|██████████| 399M/399M [00:12<00:00, 32.0MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1wslcvTNd88FQMXJgvGbKR3sjwXORR6xt
From (redirected): https://drive.google.com/uc?id=1wslcvTNd88FQMXJgvGbKR3sjwXORR6xt&confirm=t&uuid=c160edb0-a1de-4b1d-b36b-

['/content/dataset/assessing students without exams.mov',
 '/content/dataset/flipped learning basics.mov',
 '/content/dataset/teamwork in the classroom.mov']

In [8]:
# Simple Test Dataset:
paragrah_simple = "Renewable energy is crucial for reducing carbon emissions. Solar power, in particular, is sustainable and abundant. Interestingly, solar panels were first invented in 1954. With continued advancements, solar energy is becoming more accessible in everyday life."
paragraph_simple_unpunct = "Renewable energy is crucial for reducing carbon emissions  Solar power, in particular, is sustainable and abundant Interestingly, solar panels were first invented in 1954 With continued advancements, solar energy is becoming more accessible in everyday life"

# Other: CNN/Daily Mail
# dataset_news = load_dataset("cnn_dailymail", "3.0.0")
# paragraph_news = dataset_news['train']['article'][0]
# summary_news = dataset_news['train']['highlights'][0]

SRT  
each **`subtitle`** in the subtitles array has the following properties:

1. **`index`**
   - The sequential number of the subtitle within the SRT file.
   - `1`, `2`, `3`, etc. (Integer)
2. **`start`**
   - The time (in milliseconds) when the subtitle should appear on the screen.
   - `00:00:05,000` (String representing HH:MM:SS,SSS)
3. **`end`**
   - The time (in milliseconds) when the subtitle should disappear from the screen.
   - `00:00:10,000` (String representing HH:MM:SS,SSS)
4. **`content`**
   - The actual text of the subtitle that will be displayed.
   - "Hello, world!" (String)
5. **`proprietary`**
   - This field holds any additional data or formatting specific to the SRT file or software used to create it. Often empty and can usually be ignored.
   - `''` (Empty string, or sometimes contains specific formatting codes)

# Preprocessing

## Audio - Load

In [41]:
%%capture
# Extract audio (wav) from video
!ffmpeg -y -i "$video_input" -vn -acodec pcm_s16le -ar 44100 -ac 2 "$audio_output"

## Audio - SRT Generation

##### Time Taken: ~4min

In [92]:
# Select device (GPU if available, otherwise CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
language="en"
compute_type="int8"

# Model WhisperX
model = whisperx.load_model("base", device=device, language=language, compute_type=compute_type) # Choose "base" or "large" model

# Transcribe audio
result = model.transcribe(audio_output)

# Align with forced alignment
alignment_model, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], alignment_model, metadata, audio_output, device)

# Generate SRT file with aligned sentences
with open(subtitles_output, "w") as f:
    for i, segment in enumerate(result["segments"], 1):
        # Get start and end times in SRT format
        start_time = whisperx.utils.format_timestamp(segment["start"])
        end_time = whisperx.utils.format_timestamp(segment["end"])

        # Write SRT entry
        f.write(f"{i}\n{start_time} --> {end_time}\n{segment['text']}\n\n")

print(f"SRT file generated: {subtitles_output}")

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.0+cu121. Bad things might happen unless you revert torch to 1.x.
SRT file generated: dataset/teamwork in the classroom.srt


## Text - Paragraph
combination of all subtitle parts.  

WhisperAI enhances transcription with basic punctuation.

In [99]:
# Assuming aligned_segments is your WhisperX alignment output
paragraph = reduce(lambda acc, seg: acc + seg['text'].strip() + ' ', aligned_segments['segments'], '')

# Print the paragraph
print(paragraph)

Hello, this is Lino Cordia and today I want to talk to you about a very important topic, challenging topic, teamwork in the classroom. So why is teamwork in the classroom so important? Well, for our students, it allows them to develop a bunch of new skills, right? Communication skills, leadership skills, et cetera. Also, when you're working with a team, you get different perspectives. Ideally, you are part of a team that has people with different genders, people with different age groups, people with different academic backgrounds, right? So when you're talking to them, you get all these fresh perspectives that inform your the task that you're trying to solve. Also teams will motivate you, they will support you, you will feel empowered by them ideally, right? This is like the things that should happen. And also this is how the world works, right? Pretty much everything we do, we need to do it as a team. Now here's the thing about teamwork in the classroom. Students hate it. But why do 

## Text - Paragraph Summarized

In [82]:
# Model: Longformer Encoder-Decoder
model_name = "allenai/led-base-16384"
tokenizer = LEDTokenizer.from_pretrained(model_name)
model = LEDForConditionalGeneration.from_pretrained(model_name)
text = paragraph

# Tokenization
inputs = tokenizer(text, return_tensors="pt", max_length=4096, truncation=True)

# Summary Generation
summary_ids = model.generate(inputs["input_ids"], max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
paragraph_summarized = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Input ids are automatically padded from 1164 to 2048 to be a multiple of `config.attention_window`: 1024


## Text - Sentence Segmentation

In [50]:
# Segmentation
sentences = sent_tokenize(paragraph)
print(len(sentences))
print(sentences)

NameError: name 'paragraph' is not defined

# Text

In [83]:
# Simple Metrics
original_length = len(paragraph)
summary_length = len(paragraph_summarized)

print(f"original length: {original_length}")
print(f"summary length: {summary_length}")

compression_ratio = (original_length - summary_length) / original_length
print(f"auto-summary compression ratio: {compression_ratio:.2f}")

original length: 5282
summary length: 723
auto-summary compression ratio: 0.86


## Metric 1: Simple Sentence-Paragraph Relevancy (Cosine Similarity)

In [16]:
%%capture
# 1: Longformer Model
tokenizer_lf = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
model_lf = LongformerModel.from_pretrained('allenai/longformer-base-4096')

In [17]:
# 2: Tokenization
paragraph_tokens = tokenizer_lf(paragraph_summarized, return_tensors='pt')
sentence_tokens = [tokenizer_lf(sentence, return_tensors='pt') for sentence in sentences]

Embedding Explanation  
The [CLS] (classification) token is often used in transformer models to represent the overall meaning or summary of the input sequence. By extracting its embedding, you're essentially obtaining a representation that captures the main point or essence of the paragraph.

##### Time Taken: ~3min
~2.5min for 850MB video

In [18]:
# 3: Embedding
with torch.no_grad(): # Disable gradient computation for efficiency
    paragraph_embedding = model_lf(**paragraph_tokens).last_hidden_state[:, 0, :]  # Get the [CLS] token embedding
    sentence_embeddings = [model_lf(**tokens).last_hidden_state[:, 0, :] for tokens in sentence_tokens]

Input ids are automatically padded to be a multiple of `config.attention_window`: 512


KeyboardInterrupt: 

In [ ]:
# 4: Relevance scores
relevance_scores = [torch.cosine_similarity(paragraph_embedding, sentence_embedding).item() for sentence_embedding in sentence_embeddings]

# Normalization: min-max normalization
min_score = min(relevance_scores)
max_score = max(relevance_scores)
normalized_scores = [(score - min_score) / (max_score - min_score) for score in relevance_scores]

# round
normalized_scores = [np.format_float_positional(score, precision=2, unique=False, fractional=False, trim='k') for score in normalized_scores]

In [ ]:
# 5: Display Results
df = pd.DataFrame({"Sentence Index": range(len(sentences)), "Score": normalized_scores, "Sentence": sentences })
df.sort_values(by=['Score'], ascending=False, inplace=True)

display(df)

## Metric 2: Intra-sentence relevancy
Score by if current sentence is needded by adjacent sentences.

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

# Load pre-trained model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Store predictions for each sentence
predictions = []

# Iterate through sentence pairs
for i in range(len(sentences) - 1):
    sentence1 = sentences[i]
    sentence2 = sentences[i + 1]

    # Tokenize and prepare input
    inputs = tokenizer(sentence1, sentence2, return_tensors='pt', truncation=True, padding=True, add_special_tokens=True)

    # Get model prediction
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits).item()

    # Store prediction
    predictions.append(prediction)

# Handle last sentence (no next sentence)
predictions.append(0)  # Assume last sentence doesn't need a next sentence

In [ ]:
df.sort_values(by=['Sentence Index'], ascending=True, inplace=True)

# Add predictions to DataFrame
df = df.assign(**{"Previous Sentence Needed": predictions})

display(df)

## Metric 3: Intelligent Sentence-Paragraph Relevancy

##### Time Taken: 13min - 26min

In [ ]:
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096")

# Ensure the model is in evaluation mode
model.eval()

# Example usage
body_paragraph = paragraph

relevance_scores = []

for sentence in sentences:
    # Prepare the input for Longformer
    inputs = tokenizer(
        body_paragraph,
        sentence,
        return_tensors='pt',
        max_length=4096,
        truncation=True,
        padding='max_length'  # Pad to max length to avoid issues with model input size
    )

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Assuming binary classification (relevant/not relevant)
    relevance_score = torch.softmax(outputs.logits, dim=1)[0][1].item()  # Probability of being relevant
    relevance_scores.append((sentence, relevance_score))

# Sort sentences based on relevance scores
sorted_sentences = sorted(relevance_scores, key=lambda x: x[1], reverse=True)
ranked_sentences = [sentence for sentence, score in sorted_sentences]

In [ ]:
relevance_scores[0]

In [ ]:
sentence_indices = list(range(len(relevance_scores)))
scores = [score for sentence, score in relevance_scores]
sentences_text = [sentence for sentence, score in relevance_scores]

df_relevance = pd.DataFrame({'Sentence Index': sentence_indices, 'Score': scores, 'Sentence': sentences_text})
df_relevance

## Metric 4: Keyword extraction and Ranking
using TextRank

In [ ]:
# Load a spaCy model
nlp = spacy.load("en_core_web_sm")

# Add the pytextrank pipeline component to spaCy
nlp.add_pipe("textrank")

phrase_data = []

# Process the text
doc = nlp(paragraph)

for phrase in doc._.phrases:
  phrase_data.append([phrase.text, phrase.rank, phrase.count])

df_phrases = pd.DataFrame(phrase_data, columns=['Phrase', 'Rank', 'Count'])
df_phrases.sort_values(by=['Rank'], ascending=False, inplace=True)

display(df_phrases)

# Audio

## Metric 5: Silence Detection
* From the Paragraph boundaries, get the time in aduio that we care about
* For each time in audio we care about, analyze if they are low volume

OR
* analyze all potential sentence boundaries first
* match with end of sentences

In [ ]:
# 0: Load audio, extract timestamps

SAMPLING_RATE = 16000 # 16 kHz

model = load_silero_vad()
wav = read_audio(audio_output)
speech_timestamps = get_speech_timestamps(wav, model)

# Check the shape of the wav tensor
print(f"Audio shape: {wav.shape}")
print(f"Audio length (seconds): {len(wav) / SAMPLING_RATE:.2f}")

In [ ]:
# Speech Intervals
speech_intervals = []
for i in range(0, len(speech_timestamps)-1):
    speech_intervals.append((speech_timestamps[i]['start'] / SAMPLING_RATE, speech_timestamps[i+1]['end'] / SAMPLING_RATE))

# Silence Intervals
silence_intervals = []
for i in range(1, len(speech_timestamps)):
    silence_start = speech_timestamps[i-1]['end']  # End of previous speech segment
    silence_end = speech_timestamps[i]['start']     # Start of current speech segment
    silence_intervals.append((silence_start / SAMPLING_RATE, silence_end / SAMPLING_RATE))

In [ ]:
print(speech_timestamps[0:3])
print(speech_intervals[0:3])
print(silence_intervals[0:3])

In [ ]:
# Enhancing SRV
# import re

# # Split the enhanced text by sentence boundaries
# sentences = re.split(r'(?<=[.!?]) +', enhanced_text)

# # Update subtitles with enhanced sentences
# for i, subtitle in enumerate(subtitles):
#     subtitle.content = sentences[i] if i < len(sentences) else ""

# # Reconstruct the SRT
# enhanced_srt_content = srt.compose(subtitles)

# # Save the improved SRT
# with open("enhanced_subtitle_file.srt", "w") as f:
#     f.write(enhanced_srt_content)


# Video

# PostProcessing

In [ ]:
# Load the audio file
audio = AudioSegment.from_file("input_audio.wav")

# Define segments
first_part = audio[:180000]  # Up to 3:00 (in milliseconds)
second_part = audio[186000:]  # From 3:06 onward

# Combine segments
edited_audio = first_part + second_part

# Export the edited audio
edited_audio.export("output_audio.wav", format="wav")